In [3]:
import torch
import torch.nn as nn

In [34]:
class Model(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network=nn.Sequential(
        nn.Linear(num_features,3),
        nn.ReLU(),
        nn.Linear(3,1),
        nn.Sigmoid())

    def forward(self,features):
        out=self.network(features)
        return out

In [35]:
features=torch.rand(190,9)

In [36]:
model=Model(features.shape[1])

In [ ]:
model(features)

In [ ]:
model.linear.weight

In [12]:
!pip install torchinfo

In [33]:
from torchinfo import summary
summary(model,input_size=(190,9))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [190, 1]                  --
├─Linear: 1-1                            [190, 3]                  30
├─ReLU: 1-2                              [190, 3]                  --
├─Linear: 1-3                            [190, 1]                  4
├─Sigmoid: 1-4                           [190, 1]                  --
Total params: 34
Trainable params: 34
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.01
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [43]:
import kagglehub
path=kagglehub.datasets.dataset_download("yasserh/breast-cancer-dataset")
print(path)

Using Colab cache for faster access to the 'breast-cancer-dataset' dataset.
/kaggle/input/breast-cancer-dataset


In [44]:
import os
os.listdir(path)

['breast-cancer.csv']

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder,StandardScaler

In [46]:
df=pd.read_csv(path + "/breast-cancer.csv")

In [48]:
df.drop("id",inplace=True,axis=1)

In [50]:
y=df["diagnosis"]
X=df.drop("diagnosis",axis=1)

In [51]:
scaler=StandardScaler()
X_scal=scaler.fit_transform(X)
encoder=LabelEncoder()
y_en=encoder.fit_transform(y)

In [69]:
#numpy to torch.tensor
X_=torch.from_numpy(X_scal)
y_=torch.from_numpy(y_en)

In [86]:
X_=X_.float()
y_=y_.float()

In [92]:
class Model(nn.Module):
    def __init__(self,num_features):
        super().__init__()

        self.linear=nn.Linear(num_features,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,features):
        out=self.linear(features)
        out=self.sigmoid(out)
        return out

In [93]:
learning_rate=0.12
epoch=2
loss_function=nn.BCELoss()

In [94]:
##create model
model=Model(X_.shape[1])
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
#forward propogation
for i in range(epoch):
    y_pred=model(X_)
    #loss calculation
    loss=loss_function(y_pred,y_.view(-1,1))
    optimizer.zero_grad()
    #backward
    loss.backward()
    #parameter upgrade
    optimizer.step()


    print(f"epoch: {i+1} and loss = {loss.item()}")

epoch: 1 and loss = 0.6328911185264587
epoch: 2 and loss = 0.47349196672439575


In [95]:
with torch.no_grad():
    y_pred=model.forward(X_)
    y_pred=(y_pred>0.7).float()
    print(accuracy_score(y_pred,y_))

0.8734622144112478
